## Your stuff goes here ...

In [ ]:
import os
import torch
from torchvision.datasets.utils #import download_url
import zipfile

import pandas as pd

### Load data

In [4]:
# Read data - short version 3400+ documents
reuters = pd.read_pickle('input/reuters_small.pkl')
print(len(reuters))
reuters[0:5]

3426


,codes,headline,text
0,"[C18, C181, CCAT]",Eureko is latest suitor for French insurer GAN.,"\nEureko, an alliance of six European financia..."
1,"[G15, GCAT]",Reuter EC Report Long-Term Diary for July 28 -...,\n****\nHIGHLIGHTS\n****\nLUXEMBOURG - Luxembo...
2,"[G15, GCAT]",Official Journal contents - OJ L 190 of July 1...,\n*\n(Note - contents are displayed in reverse...
3,"[G15, GCAT]",Official Journal contents - OJ C 221 of July 1...,\n*\n(Note - contents are displayed in reverse...
4,"[G15, GCAT]",Official Journal contents - OJ C 220 of July 1...,\n*\n(Note - contents are displayed in reverse...


In [ ]:
# load full data, if produced. 435 MB. NOT in Github currently. 
# You can produce this in about 30 min with preprocess_data.ipynb

#reuters = pd.read_pickle('input/reuters_all.pkl')

In [63]:
# read classcodes
classcodes= pd.read_csv('input/classcodes.csv')
print(len(classcodes))
#classcodes[0:12]

126


### Using DataSet


Example in:
https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/A%20-%20Using%20TorchText%20with%20Your%20Own%20Datasets.ipynb

In [68]:
from torchtext import data
from torchtext import datasets

#Define the Fields
TEXT = data.Field()
HEADLINE = data.Field()

LABELS = data.LabelField()
#CLASSES = data.LabelField()

In [69]:
reuters[0:5]

,codes,headline,text,classes
0,"[C18, C181, CCAT]",Eureko is latest suitor for French insurer GAN.,"\nEureko, an alliance of six European financia...","[25, 26, 44]"
1,"[G15, GCAT]",Reuter EC Report Long-Term Diary for July 28 -...,\n****\nHIGHLIGHTS\n****\nLUXEMBOURG - Luxembo...,"[80, 90]"
2,"[G15, GCAT]",Official Journal contents - OJ L 190 of July 1...,\n*\n(Note - contents are displayed in reverse...,"[80, 90]"
3,"[G15, GCAT]",Official Journal contents - OJ C 221 of July 1...,\n*\n(Note - contents are displayed in reverse...,"[80, 90]"
4,"[G15, GCAT]",Official Journal contents - OJ C 220 of July 1...,\n*\n(Note - contents are displayed in reverse...,"[80, 90]"


In [70]:
len(reuters)

3426

In [71]:
### Test, save pd df to JSON
                            # one record at time, one record by line
reuters.to_json('input/reuters_small.json', orient='records', lines=True)

In [72]:
# split it
train = reuters[0:2500]
test = reuters[2500:3000]
val = reuters[3000:len(reuters)]

train.to_json('input/train.json', orient='records', lines=True)
test.to_json('input/test.json', orient='records', lines=True)
val.to_json('input/val.json', orient='records', lines=True)

In [73]:
# Tell torchText which Fields to apply to which json elements

fields = {'text': ('t', TEXT), 'headline': ('h', HEADLINE), 'classes': ('l', LABELS)}

In [74]:
fields

{'text': ('t', <torchtext.data.field.Field at 0x7fa0ef9912e8>),
 'headline': ('h', <torchtext.data.field.Field at 0x7fa0ef9916a0>),
 'classes': ('l', <torchtext.data.field.LabelField at 0x7fa0ef9916d8>)}

In [75]:
# Create dataset
train_data, valid_data, test_data = data.TabularDataset.splits(
                                        path = 'input',
                                        train = 'train.json',
                                        validation = 'val.json',
                                        test = 'test.json',
                                        format = 'json',
                                        fields = fields
)

In [76]:
# test
print(vars(train_data[0]))

{'t': ['Eureko,', 'an', 'alliance', 'of', 'six', 'European', 'financial', 'services', 'companies,', 'emerged', 'on', 'Tuesday', 'as', 'the', 'latest', 'suitor', 'for', 'ailing', 'French', 'state-controlled', 'insurer', 'GAN', 'which', 'the', 'new', 'leftist', 'government', 'plans', 'to', 'privatise.', '"Eureko', 'has', 'proposed', 'to', 'government', 'authorities', 'and', 'to', 'GAN', 'management', 'a', 'solution', 'which', 'would', 'allow', 'the', 'French', 'insurance', 'company', 'to', 'remain', 'intact', 'and', 'to', 'develop', 'its', 'national', 'and', 'international', 'position,"', 'the', 'Dutch', 'holding', 'firm', 'said.', 'The', 'Amsterdam-based', 'group,', 'which', 'has', 'no', 'presence', 'in', 'France,', 'said', 'in', 'a', 'statement', 'its', 'goal', 'was', 'to', 'take', 'a', '"strong', 'position', 'in', 'the', 'French', 'market,', 'one', 'of', 'the', 'most', 'important', 'in', 'Europe".', 'Eureko,', "Europe's", 'seventh', 'biggest', 'insurance', 'group,', 'said', 'the', 'ac